<a href="https://colab.research.google.com/github/project4sharing/exp_credit_card_fraud/blob/initial/credit_card_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project is to create a Credit Card Fraud experiment leveraging data from Kaggle .  In this journey, I try to exercise common Mahcine Learning (ML) best practice:


1.   Data wangling & cleaning
2.   Exploratory data analysis (EDA)



# High Level Summary


In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px